<a href="https://colab.research.google.com/github/alex41912/investpy/blob/main/%E5%8F%B0%E8%82%A1%E9%87%8F%E5%8C%96%E4%BA%A4%E6%98%93_%E4%BA%A4%E6%98%93%E7%AD%96%E7%95%A5%E9%96%8B%E7%99%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import numpy as np
import re
from google.colab import drive
import math
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/量化交易/")

Mounted at /content/drive


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# 用於處理輸入的函數
def run_backtest(short_window, long_window):
    # 假設我們已經有一個包含所有台灣上市櫃公司代碼的 CSV 文件 'taiwan_stock_symbols.xlsx'
    symbols_df = pd.read_excel('taiwan_stock_symbols.xlsx')

    # 設定開始和結束日期
    start_date = '2024-01-01'
    end_date = '2024-09-05'

    # 創建一個空的 DataFrame 來存儲所有公司的收盤價資料
    close_prices = pd.DataFrame()

    # 迴圈遍歷每個股票代碼並抓取資料
    for symbol in symbols_df['symbol']:
        try:
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            close_prices[symbol] = stock_data['Adj Close']  # 只選擇收盤價
        except Exception as e:
            print(f'抓取 {symbol} 收盤價資料時出錯: {e}')

    # 用於存儲每支股票的回測指標和交易記錄
    backtest_report = []
    trade_records = []

    # 對每一支股票進行回測
    for symbol in close_prices.columns:
        signals = pd.DataFrame(index=close_prices.index)

        # 計算移動平均線
        signals['short_mavg'] = close_prices[symbol].rolling(window=short_window).mean()
        signals['long_mavg'] = close_prices[symbol].rolling(window=long_window).mean()

        # 生成買賣信號
        signals['signal'] = 0.0
        signals['signal'][long_window:] = np.where(
            signals['short_mavg'][long_window:] > signals['long_mavg'][long_window:], 1.0, 0.0
        )

        # 計算買賣持倉信號
        signals['positions'] = signals['signal'].diff()

        # 初始化持倉紀錄
        current_holdings = {}

        # 模擬交易過程：當 positions 是 1 時買入，當 positions 是 -1 時賣出
        for i in range(1, len(signals)):
            if signals['positions'].iloc[i] == 1.0:
                current_holdings[symbol] = {
                    'Buy_Price': close_prices[symbol].iloc[i],
                    'Buy_Date': signals.index[i]
                }
                trade_records.append({
                    'Stock': symbol,
                    'Action': 'Buy',
                    'Date': signals.index[i],
                    'Price': close_prices[symbol].iloc[i],
                    'Return': None  # 買入時無法計算回報率
                })
            elif signals['positions'].iloc[i] == -1.0 and symbol in current_holdings:
                buy_price = current_holdings[symbol]['Buy_Price']
                sell_price = close_prices[symbol].iloc[i]
                return_rate = (sell_price - buy_price) / buy_price

                trade_records.append({
                    'Stock': symbol,
                    'Action': 'Sell',
                    'Date': signals.index[i],
                    'Price': sell_price,
                    'Return': return_rate
                })

                del current_holdings[symbol]

        # 計算每日回報
        daily_returns = close_prices[symbol].pct_change()

        # 根據買賣信號計算策略的日回報
        strategy_returns = signals['positions'].shift(1) * daily_returns
        strategy_returns.dropna(inplace=True)

        # 累積回報
        cumulative_returns = (1 + strategy_returns).cumprod() - 1

        # 計算最大回撤
        rolling_max = cumulative_returns.cummax()
        drawdown = cumulative_returns - rolling_max
        max_drawdown = drawdown.min()

        # 計算 Sharpe 比率
        sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)

        # 計算 Sortino 比率
        negative_returns = strategy_returns[strategy_returns < 0]
        downside_deviation = np.sqrt((negative_returns ** 2).mean()) * np.sqrt(252)
        sortino_ratio = strategy_returns.mean() / downside_deviation if downside_deviation != 0 else np.nan

        cumulative_return_value = cumulative_returns.iloc[-1] if not cumulative_returns.empty else np.nan

        # 將結果存儲到回測報告中
        backtest_report.append({
            'Stock': symbol,
            'Cumulative_Return': cumulative_return_value,
            'Max_Drawdown': max_drawdown,
            'Sharpe_Ratio': sharpe_ratio,
            'Sortino_Ratio': sortino_ratio
        })

    backtest_df = pd.DataFrame(backtest_report)
    trade_df = pd.DataFrame(trade_records)

    # 顯示回測報告
    display(backtest_df)
    display(trade_df)

# 創建輸入框
short_window_input = widgets.IntText(value=20, description='短期均線:')
long_window_input = widgets.IntText(value=60, description='長期均線:')
run_button = widgets.Button(description='運行回測')

# 定義按鈕點擊事件
def on_button_clicked(b):
    short_window = short_window_input.value
    long_window = long_window_input.value
    run_backtest(short_window, long_window)

# 將按鈕與事件綁定
run_button.on_click(on_button_clicked)

# 顯示輸入框和按鈕
display(short_window_input, long_window_input, run_button)

IntText(value=20, description='短期均線:')

IntText(value=60, description='長期均線:')

Button(description='運行回測', style=ButtonStyle())

FileNotFoundError: [Errno 2] No such file or directory: 'taiwan_stock_symbols.xlsx'

In [ ]:
import matplotlib.pyplot as plt

def plot_backtest_results(symbol, close_prices, signals, short_window, long_window):
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # 繪製價格走勢
    ax1.plot(close_prices[symbol], label='Price', color='blue', alpha=0.5)
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # 繪製買入信號（用綠色標記）
    ax1.plot(signals.loc[signals['positions'] == 1.0].index,
             close_prices[symbol][signals['positions'] == 1.0],
             '^', markersize=10, color='g', lw=0, label='Buy Signal')

    # 繪製賣出信號（用紅色標記）
    ax1.plot(signals.loc[signals['positions'] == -1.0].index,
             close_prices[symbol][signals['positions'] == -1.0],
             'v', markersize=10, color='r', lw=0, label='Sell Signal')

    # 創建第二個Y軸來顯示移動平均線
    ax2 = ax1.twinx()
    ax2.plot(signals['short_mavg'], label=f'Short-term MA ({short_window})', color='orange', alpha=0.7)
    ax2.plot(signals['long_mavg'], label=f'Long-term MA ({long_window})', color='green', alpha=0.7)
    ax2.set_ylabel('Moving Averages', color='green')
    ax2.tick_params(axis='y', labelcolor='green')

    # 設定標題和圖例
    fig.suptitle(f'{symbol} Price and Trading Signals')
    fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.95))
    ax1.grid(True)

    # 顯示圖表
    plt.show()

# 測試用範例資料，這裡請使用你的實際資料
plot_backtest_results('1101.TW', close_prices, signals, short_window=45, long_window=60)





NameError: name 'close_prices' is not defined

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# 假設我們已經有一個包含所有台灣上市櫃公司代碼的 CSV 文件 'taiwan_stock_symbols.xlsx'
symbols_df = pd.read_excel('taiwan_stock_symbols.xlsx')

# 設定開始和結束日期
start_date = '2024-01-01'
end_date = '2024-09-05'

# 創建一個空的 DataFrame 來存儲所有公司的收盤價資料
close_prices = pd.DataFrame()

# 迴圈遍歷每個股票代碼並抓取資料
for symbol in symbols_df['symbol']:
    try:
        # 抓取股價資料
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        close_prices[symbol] = stock_data['Adj Close']  # 只選擇收盤價
    except Exception as e:
        print(f'抓取 {symbol} 收盤價資料時出錯: {e}')

# 計算20日和60日移動平均線的交易策略
short_window = 20
long_window = 60

# 用於存儲每支股票的回測指標和交易記錄
backtest_report = []
trade_records = []
detailed_reports = {}  # 用來存儲每支股票的詳細計算過程

# 對每一支股票進行回測
for symbol in close_prices.columns:
    signals = pd.DataFrame(index=close_prices.index)

    # 計算移動平均線
    signals['short_mavg'] = close_prices[symbol].rolling(window=short_window).mean()
    signals['long_mavg'] = close_prices[symbol].rolling(window=long_window).mean()

    # 生成買賣信號
    signals['signal'] = 0.0
    signals['signal'][long_window:] = np.where(
        signals['short_mavg'][long_window:] > signals['long_mavg'][long_window:], 1.0, 0.0
    )

    # 計算買賣持倉信號
    signals['positions'] = signals['signal'].diff()

    # 初始化持倉紀錄
    current_holdings = {}

    # 模擬交易過程：當 positions 是 1 時買入，當 positions 是 -1 時賣出
    for i in range(1, len(signals)):
        if signals['positions'].iloc[i] == 1.0:
            # 買入信號，記錄該股票的買入價格和日期
            current_holdings[symbol] = {
                'Buy_Price': close_prices[symbol].iloc[i],
                'Buy_Date': signals.index[i]
            }
            trade_records.append({
                'Stock': symbol,
                'Action': 'Buy',
                'Date': signals.index[i],
                'Price': close_prices[symbol].iloc[i],
                'Return': None  # 買入時無法計算回報率
            })
        elif signals['positions'].iloc[i] == -1.0 and symbol in current_holdings:
            # 賣出信號，計算報酬率
            buy_price = current_holdings[symbol]['Buy_Price']
            sell_price = close_prices[symbol].iloc[i]
            return_rate = (sell_price - buy_price) / buy_price

            trade_records.append({
                'Stock': symbol,
                'Action': 'Sell',
                'Date': signals.index[i],
                'Price': sell_price,
                'Return': return_rate
            })

            # 清除持倉紀錄
            del current_holdings[symbol]

    # 計算每日回報
    daily_returns = close_prices[symbol].pct_change()

    # 根據買賣信號計算策略的日回報
    strategy_returns = signals['positions'].shift(1) * daily_returns  # 用 shift(1) 確保當日的信號用於次日的回報

    # 去掉NaN值
    strategy_returns.dropna(inplace=True)

    # 累積回報
    cumulative_returns = (1 + strategy_returns).cumprod() - 1

    # 計算最大回撤
    rolling_max = cumulative_returns.cummax()
    drawdown = cumulative_returns - rolling_max
    max_drawdown = drawdown.min()

    # 計算 Sharpe 比率（假設無風險利率為0）
    sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)  # 252是每年交易日

    # 計算 Sortino 比率
    negative_returns = strategy_returns[strategy_returns < 0]
    downside_deviation = np.sqrt((negative_returns ** 2).mean()) * np.sqrt(252)  # 只考慮負向波動
    sortino_ratio = strategy_returns.mean() / downside_deviation if downside_deviation != 0 else np.nan

    # 檢查cumulative_returns是否為空，避免越界錯誤
    if not cumulative_returns.empty:
        cumulative_return_value = cumulative_returns.iloc[-1]
    else:
        cumulative_return_value = np.nan  # 如果沒有累積回報資料，設為 NaN

    # 將詳細計算過程存儲起來
    detailed_reports[symbol] = pd.DataFrame({
        'Date': cumulative_returns.index,
        'Daily_Return': daily_returns,
        'Cumulative_Return': cumulative_returns,
        'Drawdown': drawdown,
        'Short_MAVG': signals['short_mavg'],
        'Long_MAVG': signals['long_mavg'],
        'Positions': signals['positions'],
        'Strategy_Return': strategy_returns
    }).set_index('Date')

    # 將結果存儲到回測報告中
    backtest_report.append({
        'Stock': symbol,
        'Cumulative_Return': cumulative_return_value,  # 總回報率
        'Max_Drawdown': max_drawdown,  # 最大回撤
        'Sharpe_Ratio': sharpe_ratio,  # Sharpe 比率
        'Sortino_Ratio': sortino_ratio  # Sortino 比率
    })

# 將回測結果轉換為 DataFrame
backtest_df = pd.DataFrame(backtest_report)
trade_df = pd.DataFrame(trade_records)

# 匯出回測結果和交易記錄到 Excel 檔案
output_file = 'backtest_detailed_report_with_trades.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # 匯出回測報告
    backtest_df.to_excel(writer, sheet_name='Backtest_Report', index=False)
    # 匯出交易記錄
    trade_df.to_excel(writer, sheet_name='Trade_Records', index=False)

    # 匯出每支股票的詳細計算過程
    for symbol, report in detailed_reports.items():
        report.to_excel(writer, sheet_name=f'{symbol}_Details')

print(f"詳細計算過程已成功匯出至 Excel 檔案：{output_file}")


串流輸出內容已截斷至最後 5000 行。
<ipython-input-8-4bed1bea256d>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  close_prices[symbol] = stock_data['Adj Close']  # 只選擇收盤價
[*********************100%***********************]  1 of 1 completed
<ipython-input-8-4bed1bea256d>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  close_prices[symbol] = stock_data['Adj Close']  # 只選擇收盤價
[*********************100%***********************]  1 of 1 completed
<ipython-input-8-4bed1bea256d>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usu

ValueError: array length 159 does not match index length 161

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from google.colab import drive
import math
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/量化交易/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import yfinance as yf
import pandas as pd
from pypfopt import EfficientFrontier, risk_models, expected_returns
import matplotlib.pyplot as plt

# 1. 抓取歷史收盤價數據
def fetch_stock_data(symbols, start_date, end_date):
    close_prices = pd.DataFrame()

    for symbol in symbols:
        try:
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            close_prices[symbol] = stock_data['Adj Close']
            print(f'{symbol} 的數據已成功抓取。')
        except Exception as e:
            print(f'抓取 {symbol} 的數據時出現錯誤: {e}')

    return close_prices

# 2. 計算資產的年化回報率與風險
def calculate_annual_return_and_risk(close_prices):
    # 計算日報酬率
    daily_returns = close_prices.pct_change().dropna()

    # 使用 PyPortfolioOpt 計算預期年化回報率和風險模型（協方差矩陣）
    mu = expected_returns.mean_historical_return(close_prices)  # 平均年化回報率
    S = risk_models.sample_cov(close_prices)  # 協方差矩陣
    return mu, S, daily_returns

# 3. 最優化資產配置（根據現代投資組合理論 MPT）
def optimize_portfolio(mu, S):
    # 使用均值-方差模型進行資產配置優化
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()  # 最大化 Sharpe 比率
    cleaned_weights = ef.clean_weights()  # 清理權重
    ef.portfolio_performance(verbose=True)  # 打印投資組合表現
    return cleaned_weights, ef

# 4. 畫出資產配置的餅狀圖
def plot_portfolio(weights):
    plt.figure(figsize=(10, 6))
    plt.pie(list(weights.values()), labels=weights.keys(), autopct='%1.1f%%', startangle=90)
    plt.title('最優化資產配置')
    plt.show()

# 主函數：抓取數據，計算，並優化投資組合
def main():
    # 從 Excel 文件中讀取台灣上市公司代碼
    symbols_df = pd.read_excel('taiwan_stock_symbols.xlsx')

    # 設定開始和結束日期
    start_date = '2024-01-01'
    end_date = '2024-09-05'

    # 提取股票代碼
    symbols = symbols_df['symbol'].tolist()

    # 抓取數據
    close_prices = fetch_stock_data(symbols, start_date, end_date)

    # 計算年化回報率與風險
    mu, S, daily_returns = calculate_annual_return_and_risk(close_prices)

    # 最優化投資組合
    optimized_weights, ef = optimize_portfolio(mu, S)

    # 畫出結果
    plot_portfolio(optimized_weights)

    # 打印權重
    print("最優化的資產配置權重：")
    print(optimized_weights)

    # 顯示日報酬
    print("每日回報率：")
    print(daily_returns.head())

# 執行主函數
if __name__ == "__main__":
    main()

# 計算日報酬的相關係數矩陣
correlation_matrix = daily_returns.corr()

# 將第一列（股票名稱）設為索引，保留相關係數的數值部分
#correlation_matrix_numeric = correlation_matrix.set_index(correlation_matrix.columns[0])

# 對於每支股票，找到與其他股票的最低相關性
min_correlation_stocks = correlation_matrix.apply(lambda row: row.idxmin(), axis=1)
min_correlation_values = correlation_matrix.apply(lambda row: row.min(), axis=1)

# 將結果組合為一個新的 DataFrame
min_correlation_df = pd.DataFrame({
    'Stock': correlation_matrix.index,
    'Min Correlation Stock': min_correlation_stocks.values,
    'Min Correlation Value': min_correlation_values.values
})

# 顯示或儲存結果
print(min_correlation_df)

# 如果希望將結果保存到 Excel 文件中
min_correlation_df.to_excel('min_correlation_stocks.xlsx', index=False)

# 顯示結果到使用者界面

# 創建一個 Excel Writer 來將多個表格寫入不同的工作表
with pd.ExcelWriter('taiwan_stock_data_and_correlation_matrix.xlsx') as writer:
    # 將原始股價資料寫入第一個工作表
    close_prices.to_excel(writer, sheet_name='Stock Prices')

    # 將日報酬資料寫入第二個工作表
    daily_returns.to_excel(writer, sheet_name='Daily Returns')

    # 將相關係數矩陣寫入第三個工作表
    correlation_matrix.to_excel(writer, sheet_name='Correlation Matrix')

    # 將每隻股票的最低相關性結果寫入第四個工作表
    min_correlation_df.to_excel(writer, sheet_name='Min Correlation Stocks')

print('股價資料、相關係數矩陣及最低相關性結果已保存到 taiwan_stock_data_and_correlation_matrix.xlsx')

z_scores = (daily_returns - daily_returns.mean()) / daily_returns.std()

# 設定 Z-score 閾值來判斷異常點，假設閾值為 3
threshold = 3

# 偵測異常點，將異常點標記為 True
anomalies = (z_scores > threshold) | (z_scores < -threshold)

# 找出有哪些股票在什麼日期出現了異常
anomalous_points = anomalies[anomalies.any(axis=1)]

# 顯示異常點
print("異常點的日期與股票如下：")
print(anomalous_points)

# 如果需要將異常點保存到 Excel 文件
anomalous_points.to_excel('anomalous_points.xlsx')

# 顯示結果
print("異常點偵測完成，已保存至 'anomalous_points.xlsx'")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


1101.TW 的數據已成功抓取。
1102.TW 的數據已成功抓取。


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


1103.TW 的數據已成功抓取。
1104.TW 的數據已成功抓取。


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

9950.TWO 的數據已成功抓取。
9951.TWO 的數據已成功抓取。


[*********************100%***********************]  1 of 1 completed
<ipython-input-11-63619e0d67da>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  close_prices[symbol] = stock_data['Adj Close']
[*********************100%***********************]  1 of 1 completed
<ipython-input-11-63619e0d67da>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  close_prices[symbol] = stock_data['Adj Close']


9960.TWO 的數據已成功抓取。
9962.TWO 的數據已成功抓取。


<ipython-input-11-63619e0d67da>:23: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = close_prices.pct_change().dropna()
/usr/local/lib/python3.10/dist-packages/pypfopt/expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
/usr/local/lib/python3.10/dist-packages/pypfopt/expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to

ValueError: Quadratic form matrices must be symmetric/Hermitian.

In [ ]:
import pandas as pd

# 讀取 Excel 文件中的數據
file_path = 'all_taiwan_stock_data.xlsx'  # 請將此路徑替換為您的實際文件路徑
df = pd.read_excel(file_path, sheet_name='Sheet1')

# 將日期列轉換為日期類型
df['Date'] = pd.to_datetime(df['Date'])

# 保留需要的欄位
df = df[['Date', 'Symbol', 'Open', 'Close', 'Adj Close', 'Volume']]

# 設置 Date 列為索引
df.set_index('Date', inplace=True)

# 按 Symbol 分組數據
grouped = df.groupby('Symbol')

# 儲存每檔股票的分析結果
results = []

# 分析每檔股票
for symbol, group in grouped:
    # 按月分組數據
    monthly_data = group.resample('M').agg({
        'Open': 'first',
        'Close': 'last',
        'Adj Close': 'last',
        'Volume': 'sum'
    }).reset_index()

    # 計算每月的收益率
    monthly_data['Return'] = (monthly_data['Close'] - monthly_data['Open']) / monthly_data['Open']

    # 計算總回報率
    total_return = (monthly_data['Return'] + 1).prod() - 1

    # 獲取開始和結束日期
    start_date = monthly_data['Date'].iloc[0]
    end_date = monthly_data['Date'].iloc[-1]
    years = (end_date - start_date).days / 365.25
    if years == 0:
        annual_return = None
        sharpe_ratio = None
    else:
        annual_return = (1 + total_return) ** (1 / years) - 1

        # 計算波動率（標準差）
        volatility = monthly_data['Return'].std() * np.sqrt(12)

        # 計算夏普比率（假設無風險利率為0）
        sharpe_ratio = annual_return / volatility

    # 儲存結果
    report = {
        'Symbol': symbol,
        '開始日期': start_date.strftime('%Y-%m-%d'),
        '結束日期': end_date.strftime('%Y-%m-%d'),
        '總回報率': f'{total_return:.2%}',
        '年化回報率': f'{annual_return:.2%}' if annual_return is not None else 'N/A',
        '波動率': f'{volatility:.2%}' if annual_return is not None else 'N/A',
        '夏普比率': f'{sharpe_ratio:.2f}' if sharpe_ratio is not None else 'N/A'
    }
    results.append(report)

# 轉換結果為 DataFrame
results_df = pd.DataFrame(results)

# 顯示 DataFrame
print(results_df)


<ipython-input-26-9aa639bc8212>:52: RuntimeWarning: invalid value encountered in scalar divide
  sharpe_ratio = annual_return / volatility


        Symbol        開始日期        結束日期     總回報率    年化回報率     波動率   夏普比率
0      0050.TW  2024-01-31  2024-08-31   28.80%   54.34%  18.94%   2.87
1      0051.TW  2024-01-31  2024-08-31    6.35%   11.13%  11.76%   0.95
2      0052.TW  2024-01-31  2024-08-31   31.63%   60.20%  22.85%   2.63
3      0053.TW  2024-01-31  2024-08-31   30.89%   58.65%  21.56%   2.72
4      0055.TW  2024-01-31  2024-08-31   17.99%   32.81%  10.64%   3.08
...        ...         ...         ...      ...      ...     ...    ...
1970  9951.TWO  2024-01-31  2024-08-31  -13.66%  -22.27%  12.23%  -1.82
1971   9955.TW  2024-01-31  2024-08-31   37.92%   73.55%  39.02%   1.89
1972   9958.TW  2024-01-31  2024-08-31   30.47%   57.78%  64.67%   0.89
1973  9960.TWO  2024-01-31  2024-08-31   -5.39%   -9.07%   8.96%  -1.01
1974  9962.TWO  2024-01-31  2024-08-31  -17.56%  -28.18%   8.93%  -3.16

[1975 rows x 7 columns]


In [ ]:
df_sorted

,Symbol,開始日期,結束日期,總回報率,年化回報率,波動率,夏普比率
1181,4950.TWO,2024-01-31,2024-08-31,0.00%,0.00%,0.00%,nan
1710,6838.TW,2024-08-31,2024-08-31,-16.34%,N/A,N/A,N/A
116,00954.TW,2024-08-31,2024-08-31,-2.30%,N/A,N/A,N/A
117,00955.TWO,2024-08-31,2024-08-31,1.30%,N/A,N/A,N/A
118,00956.TW,2024-08-31,2024-08-31,1.10%,N/A,N/A,N/A
...,...,...,...,...,...,...,...
390,2106.TW,2024-01-31,2024-08-31,-0.07%,-0.11%,15.63%,-0.01
1753,7402.TWO,2024-01-31,2024-08-31,-0.22%,-0.38%,61.89%,-0.01
1603,6591.TW,2024-01-31,2024-08-31,-0.18%,-0.31%,35.49%,-0.01
329,1795.TW,2024-01-31,2024-08-31,-0.22%,-0.37%,31.67%,-0.01


In [ ]:
results_df.to_excel('股票回測.xlsx')

In [ ]:
report_df.to_excel('1101.xlsx')